In [1]:
import numpy as np
import pandas as pd
import sympy as sp
from fractions import Fraction
import itertools
import timeit

### Funciones previas

In [2]:
def permutation(lst):
    
    if len(lst) == 0:
        return []
    if len(lst) == 1:
        return [lst]
    
    l = []

    for i in range(len(lst)):
        m = lst[i]
        remLst = lst[:i] + lst[i+1:]

        for p in permutation(remLst):
            l.append([m] + p)
        
    return l

# Paridad de las permutaciones (solo funciona con el orden de la funcion anterior)

def par(lst):
    
    l = []
    
    for i in range(len(lst)):
        if i % 2 == 0:
            l.append(1)
            
        else:
            l.append(-1)
    return np.cumprod(np.array(l))

def combinations(n,k):
    
    u = list(range(0,n))
    
    c = [list(v) for v in itertools.product(u, repeat=k) if len([i for i in v if v.count(i)>1])==0]
    
    return c 
 
def kCombinations(output, temp, index, n, i, K):
    if index == K:
        output.append(temp.copy())
        return
    
    if i >=n:
        return
    
    temp[index] = i #+ 1
    kCombinations(output, temp, index + 1, n, i + 1, K)
    kCombinations(output, temp, index, n, i + 1, K)
    
def findKCombination(K,n):
    
    output = []
    temp = [0]*K
    kCombinations(output, temp, 0, n, 0, K)
    return output

### Delta Generalizada

In [3]:
def Delta(upI,lwI):
    
    per = permutation(lwI)
    pari = par(per)
    
    # Verifica si hay indices repetidos
    
    if len([i for i in upI if upI.count(i)>1]) > 0:
        return 0
    if len([i for i in lwI if lwI.count(i)>1]) > 0:
        return 0  
    
    # Busca si los indices superiores son una permutacion (par o impar) del los inferiores
    
    if upI in per:
        k = per.index(upI)
        return sp.Rational(pari[k],np.math.factorial(len(upI)))
    else:
        return 0

### Check de traza

Deberia ser 

$\Delta^{\mu_1 ... \mu_k}_{\nu_1 ... \nu_k} = \frac{1}{k!}\frac{n!}{(n-k)!}$

In [4]:
def TraDelta(n,k):
    u = 0
    for i in combinations(n,k):
        u = u + Delta(i,i)
    return u  

def checkDelta(n,k):
    
    s = sp.Rational(1,np.math.factorial(k))*sp.Rational(np.math.factorial(n),np.math.factorial(n-k))
    
    if TraDelta(n,k) == s:
        return True
    else: return False
        

In [5]:
checkDelta(4,1)

True

In [6]:
checkDelta(4,2)

True

In [7]:
checkDelta(4,3)

True

In [8]:
checkDelta(5,2)

True

In [9]:
checkDelta(5,3)

True

In [10]:
checkDelta(5,4)

True

### Representacion fundamental de SU(n)

In [11]:
def Li(n,i,j,a,b):
    
    s = Delta([a],[j])*Delta([i],[b])-Delta([i],[j])*Delta([a],[b])*sp.Rational(1,n)
    
    return s

def LiM(n,i,j):
    
    u = np.zeros((n,n),dtype=object)
    
    for a in range(0,n):
        for b in range(0,n):
            u[a][b]=Li(n,i,j,a,b)
    return u   

def L2(n,i,j):
    
    u = np.zeros((n,n),dtype=object)
    
    for a in range(0,n):
        for b in range(a,n):
            u[a][b] = (sp.Rational(1,2))*(Li(n,i,j,a,b)+Li(n,j,i,a,b))
            u[b][a] = u[a][b]
            
    return u          

def M2(n,i,k):
    
    u = np.zeros((n,n),dtype=object)
    
    for a in range(0,n):
        for b in range(a,n):
            u[a][b] = (sp.Rational(1,2))*(1j)*(Li(n,i,k,a,b)-Li(n,k,i,a,b))
            u[b][a] = -u[a][b]
            
    return u

def D(n,k):
    
    u = sp.sqrt(2*(k+1)*(k+2))
    
    s = np.zeros((n,n),dtype=object)
    
    for i in range(0,k+1):
        s = s + LiM(n,i,i)
        
    s = s - (k+1)*LiM(n,k+1,k+1)
    
    return s*(1/u)

In [12]:
def GenLamda3(n):
    
    p = []
    b = []

    for j in range(1,n):
        for i in range(0,j):
           # if i >= j:
           #     continue    

            p.append(2*L2(n,i,j))
            b.append('s')
            p.append(2*M2(n,i,j))
            b.append('as')
        p.append(2*D(n,j-1))
        b.append('d')
    return p,b

### Traza de los generadores $\lambda_I$

In [13]:
def Traza(A,B):
    
    u = np.trace(np.matmul(A,B,dtype=object))
    
    return u

In [14]:
def checkTL(Lamdas):
    
    u = (Lamdas[0].shape)[0]
    s = []
    
    for i,j in itertools.product(range(0,u*u-1), repeat=2):
        
        if Traza(Lamdas[i],Lamdas[j]) == 2*Delta([i],[j]):
            s.append(True)
        else:
            s.append(False)
    
    
    if all(s) == True:
        return 'Todas las relaciones se cumplen'
    else:
        return 'Algo va mal'
    
    

In [15]:
checkTL(GenLamda3(2)[0])

'Todas las relaciones se cumplen'

In [16]:
checkTL(GenLamda3(3)[0])

'Todas las relaciones se cumplen'

In [17]:
checkTL(GenLamda3(4)[0])

'Todas las relaciones se cumplen'

In [18]:
checkTL(GenLamda3(5)[0])

'Todas las relaciones se cumplen'

In [19]:
checkTL(GenLamda3(6)[0])

'Todas las relaciones se cumplen'

### Calculo de los $t_I$

$(t_I)^{\mu_1 \ldots \mu_k}_{\nu_1 \ldots \nu_k} = (-1)^{k-1}(k-1)!\delta_{l_1}^{[\mu_1}\Delta^{\mu_2 \ldots \mu_k]\rho_1}_{\nu_1 \ldots \nu_k}(\lambda_I)_{\rho_1}^{l_1}$

In [20]:
def Pro(UpI,LwI):
    
    s = 0
    
    perU = permutation(UpI[:len(UpI)-1])
    pari = par(perU)
    
    if len([i for i in UpI[:len(UpI)-1] if UpI[:len(UpI)-1].count(i)>1]) > 0:
        return 0
    else:
        for i in perU:
            k = perU.index(i)
            s = s + Delta([i[0]],[LwI[0]])*Delta(i[1:]+[UpI[-1]],LwI[1:])*pari[k]*Fraction(1,np.math.factorial(len(UpI)-1))
    return s 

In [21]:
def t3(n,matrix,upI,lwI):
    
    i = 0
    
    sig = ((-1)**(len(upI)-1))*np.math.factorial(len(upI)-1)
    
    s,t = np.where(matrix !=0 )
    
    
    for p,m in zip(s,t):
        UpI = upI + [p]
        LpI = [m] + lwI
        i = i + Pro(UpI,LpI)*(matrix)[p][m]
    
    return i*sig

### Verifiquemos la traza

In [54]:
def Tract(matrix,k):
    
    u = (matrix[0].shape)[0]
    
    p = combinations(u,k)
    
    rs = 2*sp.Rational(np.math.factorial(k-1)*np.math.factorial(u-2),k*k*np.math.factorial(u-k-1))
    
    t = []
    
    for i,j in itertools.product(range(0,u*u-1), repeat=2):
        
        l = []
        
        for upI in p:
            for lwI in p:
                l.append(t3(u,matrix[i],upI,lwI)*t3(u,matrix[j],lwI,upI))
                
        #print(sp.nsimplify(sum(l)),sp.nsimplify(rs*Delta([i],[j])))
        
        if sp.nsimplify(sum(l)) == sp.nsimplify(rs*Delta([i],[j])):
            t.append(True)
        else:
            t.append(False)
            
    if all(t) == True:
        return 'Todas las relaciones se cumplen'
    else:
        return 'Algo va mal'            

In [55]:
Tract(GenLamda3(3)[0],2)

'Todas las relaciones se cumplen'

In [56]:
Tract(GenLamda3(4)[0],2)

'Todas las relaciones se cumplen'

In [58]:
Tract(GenLamda3(4)[0],3)

'Todas las relaciones se cumplen'

In [59]:
Tract(GenLamda3(5)[0],2)

'Todas las relaciones se cumplen'

### Veamos el bracket

In [110]:
def brat(n,A,B,upI,lwI):
    
    k = len(upI)
    u = 0
    
    p = combinations(n,k)
        
    for s in p:
        u = u + (t3(n,A,s,lwI)*t3(n,B,upI,s) - t3(n,B,s,lwI)*t3(n,A,upI,s))
        
    return u    
    

In [124]:
sp.nsimplify(brat(5,(GenLamda3(5)[0])[0],(GenLamda3(5)[0])[1],[0,2,3,4],[0,2,3,4]))

3*I/16

In [125]:
t3(4,(GenLamda3(5)[0])[2],[0,2,3,4],[0,2,3,4])

1/16

In [ ]:
def rhstbra(n,st,upI,lwI):
    
    k
    
    u = 0
    
    for i in range(0,)
    
    

## Representacion Matricial de los generadores

In [64]:
def tM(n,Ma,st,k):
    
    values = findKCombination(k,n)
    dim = len(values)
    keys = [i for i in range(0,dim)]
    
    dic = dict(zip(keys,values))
    
    matrix = np.zeros((dim,dim),dtype=object)
    
    if st == 's':
        
        for i in range(0,dim):
            for j in range(i,dim):
                matrix[i][j] = t3(n,Ma,dic[j],dic[i])
                matrix[j][i] = matrix[i][j]
                
    elif st == 'as':
        
        for i in range(0,dim):
            for j in range(i,dim):
                matrix[i][j] = t3(n,Ma,dic[j],dic[i])
                matrix[j][i] = -matrix[i][j]
                
    elif st == 'd':
        
        for i in range(0,dim):
            matrix[i][i] = t3(n,Ma,dic[i],dic[i])       
            
    return matrix

In [79]:
def tMf(n,k):
    
    ##############################################################
    ############ Todas las matrices y su simetria ################
    ##############################################################
    
    Lamda, Sim = GenLamda3(n)
    
    ###############################################################
    ######## Representacion Matricial de los T ####################
    ###############################################################
    
    a = []
    
    for u in range(0,n*n-1):
        
        a.append(tM(n,Lamda[u],Sim[u],k))
    
    return Lamda,a

In [80]:
def bra(A,B):
    
    return np.matmul(A,B,dtype=object)-np.matmul(B,A,dtype=object)

### Anticomutador


def Abra(A,B):
    
    return np.matmul(A,B,dtype=object)+np.matmul(B,A,dtype=object)



def f3(LamdL,n):
       
    u = range(0,n*n-1)
    
   # values = [list(v) for v in itertools.product(u, repeat=3)]
    di = {}
    
    for i,j,k in itertools.product(u, repeat=3):
        
        m = sp.nsimplify(-(1j)*sp.Rational(1,4)*np.trace(np.matmul(LamdL[i],bra(LamdL[j],LamdL[k]),dtype=object)))
        di[(i,j,k)] = m
        
    return di    



## Generadores $T_I$ 

En esta parte se calculan todos los $T_I$ en su representacion matricial. Por simplicidad aqui calculare todos los generadores para $n=3$,$n=4$,$n=5$. Para todos los valores de $2 \leq k < n$.

### n=3

In [81]:
# k=2

Lamda_3, T32 = tMf(3,2)

In [82]:
F_3 = f3(Lamda_3,3)

### n=4

In [127]:
# k = 2

Lamda_4, T42 = tMf(4,2) 

# k =3
T43 = tMf(4,3)[1]

In [128]:
F_4 = f3(Lamda_4,4)

### n = 5

In [129]:
# k = 2

Lamda_5, T52 = tMf(5,2)

# k = 3

T53 = tMf(5,3)[1]

# k = 4

T54 = tMf(5,4)[1]

In [130]:
F_5 = f3(Lamda_5,5)

In [87]:
T42[2]

array([[0, 0, 0, 0, 0, 0],
       [0, 1/4, 0, 0, 0, 0],
       [0, 0, 1/4, 0, 0, 0],
       [0, 0, 0, -1/4, 0, 0],
       [0, 0, 0, 0, -1/4, 0],
       [0, 0, 0, 0, 0, 0]], dtype=object)

### Pruebas de traza y algebra

In [74]:
## Comutador



## Constantes de estructura de SU(n)

def f2(LamdL,I,J,K):
    
    if I == J or J == K:
        return 0
    else:
        return  sp.nsimplify(-(1j)*sp.Rational(1,4)*np.trace(np.matmul(LamdL[I],bra(LamdL[J],LamdL[K]),dtype=object)))
    


### Algebra

In [131]:
def rhs(n,I,J,Mat,Lamda,st):
    
    a = np.zeros((len(Mat[0]),len(Mat[0])),dtype=object)
    
    for i in range(0,n*n-1):
        if st == 'l':
            p = 2j
        elif st == 't':
            p = 2j
        elif st == 'T':
            p = 1
        u = f2(Lamda,I,J,i)*p*Mat[i]
        u[u!=0] = np.array([sp.nsimplify(i) for i in u[u!=0]],dtype=object)
        
        a = a + u
        
    return a

def rhs2(n,I,J,Mat,estc,st,k):
    
    a = np.zeros((len(Mat[0]),len(Mat[0])),dtype=object)
    
    for i in range(0,n*n-1):
        if st == 'l':
            p = 2j
        elif st == 't':
            p = sp.nsimplify(2j*sp.Rational(1,k*k))
        elif st == 'T':
            p = 1
        u = estc[(I,J,i)]*p*Mat[i]
        u[u!=0] = np.array([sp.nsimplify(i) for i in u[u!=0]],dtype=object)
        
        a = a + u
        
    return a
    

In [90]:
def alg(n,I,J,Mat,Lamda,st,k):
    
    a = rhs(n,I,J,Mat,Lamda,st)
        
    b = bra(Mat[I],Mat[J])
    
    ## Verifiquemos que todo sea del mismo tipo
    
    a[a!=0] = np.array([sp.nsimplify(i) for i in a[a!=0]],dtype=object)
    b[b!=0] = np.array([sp.nsimplify(i) for i in b[b!=0]],dtype=object)
    
    if (b == a).all():
        return True
    else:
        return False
    
def alg2(n,I,J,Mat,estc,st,k):
    
    a = rhs2(n,I,J,Mat,estc,st,k)
        
    b = bra(Mat[I],Mat[J])
    
    ## Verifiquemos que todo sea del mismo tipo
    
    a[a!=0] = np.array([sp.nsimplify(i) for i in a[a!=0]],dtype=object)
    b[b!=0] = np.array([sp.nsimplify(i) for i in b[b!=0]],dtype=object)
    
    if (b == a).all():
        return True
    else:
        return False    

### Vamos ahora con las pruebas para cada caso

In [91]:
def Algtest(n,Mat,Lamda,st,k):
    
    a = []
    
    for i,j in combinations(n*n-1,2):
        a.append(alg(n,i,j,Mat,Lamda,st))
        
        # Para saber en que indices no se cumple
        
        # if alg(n,i,j,Mat,st) == False:
            
        #     print(i,j)
        
    if all(a) == True:
        return 'Todas las relaciones se cumplen'
    else:
        return 'Algo va mal'
    
def Algtest2(n,Mat,estc,st,k):
    
    a = []
    
    for i,j in combinations(n*n-1,2):
        a.append(alg2(n,i,j,Mat,estc,st,k))
        
        # Para saber en que indices no se cumple
        
        #if alg2(n,i,j,Mat,estc,st) == False:
            
        #    print(i,j)
        
    if all(a) == True:
        return 'Todas las relaciones se cumplen'
    else:
        return 'Algo va mal'    

### Verificando las relaciones de conmutacion de los $\lambda_I$

In [93]:
Algtest2(3,Lamda_3,F_3,'l',1)

'Todas las relaciones se cumplen'

In [94]:
Algtest2(4,Lamda_4,F_4,'l',1)

'Todas las relaciones se cumplen'

In [95]:
Algtest2(5,Lamda_5,F_5,'l',1)

'Todas las relaciones se cumplen'

### Verificando las relaciones de commutacion de los $t_I$

In [135]:
Algtest2(3,T32,F_3,'t',2)

'Todas las relaciones se cumplen'

In [136]:
Algtest2(4,T42,F_4,'t',2)

'Todas las relaciones se cumplen'

In [137]:
Algtest2(4,T43,F_4,'t',3)

'Todas las relaciones se cumplen'

In [139]:
Algtest2(5,T52,F_5,'t',2)

'Todas las relaciones se cumplen'

In [140]:
Algtest2(5,T53,F_5,'t',3)

'Todas las relaciones se cumplen'

In [141]:
Algtest2(5,T54,F_5,'t',4)

'Todas las relaciones se cumplen'

In [142]:
bra(T43[0],T43[1])

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0.0246913580246914*I, 0],
       [0, 0, 0, -0.0246913580246914*I]], dtype=object)

In [134]:
rhs2(4,0,1,T43,F_4,'t',3)

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 2*I/81, 0],
       [0, 0, 0, -2*I/81]], dtype=object)

In [59]:
bra(T53[0],T53[2])

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, -8/81, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, -8/81, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, -8/81, 0],
       [0, 0, 0, 8/81, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 8/81, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 8/81, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=object)

In [60]:
rhs2(5,0,2,T53,F_5,'t')

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, -2/9, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, -2/9, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, -2/9, 0],
       [0, 0, 0, 2/9, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 2/9, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 2/9, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=object)

### Veamos por que 

### Verificando la traza

In [37]:
def traz(A,B):
    
    u = np.trace(np.matmul(A,B,dtype=object))
    
    return u

In [ ]:
def checkTrace(ListM,strig):
    
    
    